In [1]:
from datasets import load_dataset

ds = load_dataset("KShivendu/dbpedia-entities-openai-1M")

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['_id', 'title', 'text', 'openai'],
        num_rows: 1000000
    })
})

In [7]:
ds["train"].to_parquet("dbpedia-entities-openai-1M-train.parquet")

Creating parquet from Arrow format:   0%|          | 0/1000 [00:00<?, ?ba/s]

6535725776

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("data/dbpedia-entities-openai-1M-train.parquet")

In [5]:
df.shape

(1000000, 4)

In [3]:
records = df.rename(columns={"_id": "item_id"}).to_dict(orient="records")


In [4]:
len(records[0]["openai"])

1536

In [5]:
from redisvl.redis.utils import array_to_buffer

parsed_records = [
    {
        "item_id": r["item_id"],
        "text": r["text"],
        "title": r["title"],
        "text_embedding": array_to_buffer(r["openai"], dtype="float32"),
    } for r in records
]

In [19]:
jsonable = [{"item_id": r["item_id"], "text": r["text"], "title": r["title"]} for r in parsed_records]

In [20]:
import json

no_embedding = []

with open("dbpedia-entities-openai-1M-train.json", "w") as f:
    json.dump(jsonable, f)

In [6]:
import json

with open("data/dbpedia_250k.json", "r") as f:
    dbpedia_250k = json.load(f)

In [7]:
from redis import Redis
from redisvl.index import SearchIndex

client = Redis.from_url("redis://localhost:6379/0")
index = SearchIndex.from_yaml("schema.yaml")
index.set_client(client)
index.create()


In [8]:
index.load(dbpedia_250k, id_field="item_id")

09:11:17 redisvl.index.index ERROR   Error while loading data to Redis
Traceback (most recent call last):
  File "/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/redisvl/index/index.py", line 570, in load
    return self._storage.write(
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/redisvl/index/storage.py", line 230, in write
    pipe.execute()
  File "/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/redis/client.py", line 1530, in execute
    return conn.retry.call_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/redis/retry.py", line 62, in call_with_retry
    return do()
           ^^^^
  File "/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/redis/client.py", line 1531, in <lambda>
    lambda: execute(conn, stack, raise_on_error),
            ^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [38]:
info = index.create()

In [9]:
info = index.info()

In [15]:
float(info["total_indexing_time"])

1292258.125

In [42]:
import time

while float(index.info()["percent_indexed"]) < 0.9:
    time.sleep(5)
    info = index.info()
    print(f"\n num_docs: {info['num_docs']}, percent_indexed: {info['percent_indexed']}")


 num_docs: 88110, percent_indexed: 0.08810953293307787

 num_docs: 89136, percent_indexed: 0.08913550420588223

 num_docs: 90059, percent_indexed: 0.09005847836260585

 num_docs: 90993, percent_indexed: 0.09099345218333886

 num_docs: 91924, percent_indexed: 0.09192442611606876

 num_docs: 92923, percent_indexed: 0.09292339814485194

 num_docs: 93904, percent_indexed: 0.09390437067762103

 num_docs: 94849, percent_indexed: 0.09485034419036267

 num_docs: 95816, percent_indexed: 0.09581731711512077

 num_docs: 96736, percent_indexed: 0.09673729135584204

 num_docs: 97727, percent_indexed: 0.09772826360861896

 num_docs: 98663, percent_indexed: 0.09866423740135276

 num_docs: 99629, percent_indexed: 0.09963021035411009

 num_docs: 100618, percent_indexed: 0.10061918266288544

 num_docs: 101580, percent_indexed: 0.10158115572763962

 num_docs: 102530, percent_indexed: 0.1025311291283844

 num_docs: 103434, percent_indexed: 0.10343510381709312

 num_docs: 104343, percent_indexed: 0.104344

KeyboardInterrupt: 

In [ ]:
import os
from redisvl.utils.vectorize import OpenAITextVectorizer

os.environ["OPENAI_API_KEY"] = ""

emb_model = OpenAITextVectorizer("text-embedding-ada-002")

In [36]:
emb = emb_model.embed("Hello, world!")

In [37]:
len(emb)

1536

In [9]:
METRICS = {
    "ret_k": [],
    "algorithm": [],
    "ef_construction": [],
    "ef_runtime": [],
    "m": [],
    "distance_metric": [],
    "vector_data_type": [],
    "model": [],
    "model_dim": [],
    "f1@k": [],
    "embedding_latency": [],
    "indexing_time": [],
    "avg_query_latency": [],
    "obj_val": [],
    "retriever": [],
}

import pandas as pd

pd.DataFrame(METRICS).to_csv("data/metrics.csv", index=False)